In [20]:
import numpy as np
import pandas as pd
import bz2
import pickle
import _pickle as cPickle
import csv

In [26]:
# load data
all_movies = pd.read_csv("all_movies.csv")


# remove duplicates if any
all_movies = all_movies.drop_duplicates(subset='movie_id', keep='last')

# all_movies = all_movies[all_movies['movie_id'].notna()]
# all_movies = all_movies[all_movies['Title'].notna()]
# all_movies = all_movies[all_movies['cast'].notna()]
# all_movies = all_movies[all_movies['crew'].notna()]
# all_movies = all_movies[all_movies['genres'].notna()]
all_movies = all_movies.dropna()

all_movies['movie_id'] = all_movies['movie_id'].astype(int)

# all_movies = all_movies.sort_values(by=['Title'])
all_movies

,Title,cast,crew,genres,final_tag,movie_id
0,Avatar,SamWorthington ZoeSaldana,JamesCameron,Action Adventure Fantasy ScienceFiction,"In the 22nd century, a paraplegic Marine is di...",19995
1,Pirates of the Caribbean: At World's End,JohnnyDepp OrlandoBloom,GoreVerbinski,Adventure Fantasy Action,"Captain Barbossa, long believed to be dead, ha...",285
2,Spectre,DanielCraig ChristophWaltz,SamMendes,Action Adventure Crime,A cryptic message from Bond’s past sends him o...,206647
3,The Dark Knight Rises,ChristianBale MichaelCaine,ChristopherNolan,Action Crime Drama Thriller,Following the death of District Attorney Harve...,49026
4,John Carter,TaylorKitsch LynnCollins,AndrewStanton,Action Adventure ScienceFiction,"John Carter is a war-weary, former military ca...",49529
...,...,...,...,...,...,...
6216,The Woman King,Viola Davis,Gina Prince-Bythewood,Drama History,A historical epic inspired by true events that...,724495
6217,"Are You There God? It's Me, Margaret",Abby Ryder Fortson,Kelly Fremon Craig,Drama Comedy,When her family moves from the city to the sub...,555285
6218,Distant,Anthony Ramos,Josh Gordon & Will Speck,Western,"After destroying a Seminole fort, American sol...",61391
6219,Don't Worry Darling,Florence Pugh,Olivia Wilde,Drama Mystery Thriller,A housewife living in a utopian community in t...,619730


In [37]:
all_movies = all_movies.reset_index(drop=True)
all_movies

,Title,cast,crew,genres,final_tag,movie_id
0,Avatar,SamWorthington ZoeSaldana,JamesCameron,Action Adventure Fantasy ScienceFiction,"in the 22nd century, a parapleg marin is dispa...",19995
1,Pirates of the Caribbean: At World's End,JohnnyDepp OrlandoBloom,GoreVerbinski,Adventure Fantasy Action,"captain barbossa, long believ to be dead, ha c...",285
2,Spectre,DanielCraig ChristophWaltz,SamMendes,Action Adventure Crime,a cryptic messag from bond’ past send him on a...,206647
3,The Dark Knight Rises,ChristianBale MichaelCaine,ChristopherNolan,Action Crime Drama Thriller,follow the death of district attorney harvey d...,49026
4,John Carter,TaylorKitsch LynnCollins,AndrewStanton,Action Adventure ScienceFiction,"john carter is a war-weary, former militari ca...",49529
...,...,...,...,...,...,...
6010,The Woman King,Viola Davis,Gina Prince-Bythewood,Drama History,a histor epic inspir by true event that took p...,724495
6011,"Are You There God? It's Me, Margaret",Abby Ryder Fortson,Kelly Fremon Craig,Drama Comedy,when her famili move from the citi to the subu...,555285
6012,Distant,Anthony Ramos,Josh Gordon & Will Speck,Western,"after destroy a seminol fort, american soldier...",61391
6013,Don't Worry Darling,Florence Pugh,Olivia Wilde,Drama Mystery Thriller,a housewif live in a utopian commun in the cal...,619730


In [38]:
print(all_movies[all_movies['Title'] == 'Bros'].index[0])

6014


In [39]:
# remove similar words, stem the words
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stem(text):
    newList = []
    for words in text.split():
        newList.append(ps.stem(words))
    return " ".join(newList)

all_movies['final_tag'] = all_movies['final_tag'].apply(stem)

In [40]:
# create the model
from sklearn.feature_extraction.text import CountVectorizer
# vectorize
cv1 = CountVectorizer(max_features=25000, stop_words='english')
cv2 = CountVectorizer(max_features=5000, stop_words='english')
cv3 = CountVectorizer(max_features=4500, stop_words='english')
cv4 = CountVectorizer(max_features=5, stop_words='english')
vector1 = cv1.fit_transform(all_movies['final_tag'].apply(lambda x: np.str_(x)))
vector2 = cv2.fit_transform(all_movies['cast'].apply(lambda x: np.str_(x)))
vector3 = cv3.fit_transform(all_movies['crew'].apply(lambda x: np.str_(x)))
vector4 = cv4.fit_transform(all_movies['genres'].apply(lambda x: np.str_(x)))

In [41]:
print(vector1[0].shape)
print(vector2[0].shape)
print(vector3[0].shape)
print(vector4[0].shape)

(1, 25000)
(1, 5000)
(1, 4319)
(1, 5)


In [42]:
# calculate cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

similarity1 = cosine_similarity(vector1)
similarity2 = cosine_similarity(vector2)
similarity3 = cosine_similarity(vector3)
similarity4 = cosine_similarity(vector4)


In [43]:
similarity1.shape

(6015, 6015)

In [44]:
all_movies[all_movies['Title'] == np.nan]

,Title,cast,crew,genres,final_tag,movie_id


In [45]:
all_movies[all_movies['Title'] == "Batman Begins"]

,Title,cast,crew,genres,final_tag,movie_id
116,Batman Begins,ChristianBale MichaelCaine,ChristopherNolan,Action Crime Drama,"driven by tragedy, billionair bruce wayn dedic...",272


In [46]:
all_movies[all_movies['Title'] == '1917']

,Title,cast,crew,genres,final_tag,movie_id
5391,1917,Krysty Wilson-Cairns (screenplay); George MacKay,Sam Mendes,War Drama Action Thriller,"at the height of the first world war, two youn...",530915


In [47]:
# main recommendation part
def recommend_movies(movie):
    temp = str(movie)
    temp = temp.split()
    if len(temp) > 1:
        if temp[0] in ['A', 'An', 'The'] and len(temp) == 2:
            temp = temp[1]
        elif len(temp) > 2 and temp[0] not in ['A', 'An', 'The']:
            temp = ' '.join(temp[:2])
        else:
            temp = ' '.join(temp)
    else:
        temp = temp[0]
            
    recommended_list = []
    for index, row in all_movies.iterrows():
        curr = row[0]
        if temp in curr and movie != curr:
            recommended_list.append(curr)
            print(curr)
   
    # calculate the index of the movie
    index = all_movies[all_movies['Title'] == movie].index[0]
    
    print("\nbased on tag ---------------------------------")
    
    dist1 = similarity1[index]
    distances = sorted(list(enumerate(dist1)),reverse=True,key = lambda x: x[1])
    for i in distances[0:6]:
        curr = all_movies.iloc[i[0]].Title
        if curr != movie:
            print(curr)
            if curr not in recommended_list:
                recommended_list.append(curr)
    
    print("\nbased on genre ---------------------------------")
    
    dist4 = similarity4[index]
    distances = sorted(list(enumerate(dist4)),reverse=True,key = lambda x: x[1])
    for i in distances[0:6]:
        curr = all_movies.iloc[i[0]].Title
        if curr != movie:
            print(curr)
            if curr not in recommended_list:
                recommended_list.append(curr)
    
    print("\nbased on cast -----------------------------------")
    # distances
    dist2 = similarity2[index]
    distances = sorted(list(enumerate(dist2)),reverse=True,key = lambda x: x[1])
    for i in distances[0:6]:
        curr = all_movies.iloc[i[0]].Title
        if curr != movie:
            print(curr)
            if curr not in recommended_list:
                recommended_list.append(curr)
    
    print("\nbased on director ---------------------------------")
    
    dist3 = similarity3[index]
    distances = sorted(list(enumerate(dist3)),reverse=True,key = lambda x: x[1])
    for i in distances[0:6]:
        curr = all_movies.iloc[i[0]].Title
        if curr != movie:
            print(curr)
            if curr not in recommended_list:
                recommended_list.append(curr)
        
    

In [53]:
# recommend_movies("Pirates of the Caribbean: Dead Man's Chest")
# recommend_movies("Interstellar")
# recommend_movies("Fifty Shades of Grey")
# recommend_movies("Schindler's List")
# recommend_movies("The Terminator")
recommend_movies("1917")
# recommend_movies("The Dark Knight Rises")



based on tag ---------------------------------
Restless
Ghosts of War
Cosmic Sin
The 355
Saints and Soldiers

based on genre ---------------------------------
The Dark Knight Rises
World War Z
The Dark Knight
Dawn of the Planet of the Apes
Poseidon
I Am Legend

based on cast -----------------------------------
Rock Dog
Secret Headquarters
Becoming Bond
Cars 3
Father Figures

based on director ---------------------------------
Humor Me
Assassination Nation
Extraction
Guest House
Malcolm & Marie


In [54]:
# import pickle

# Pickle a file and then compress it into a file with extension 
def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'wb') as f:
        cPickle.dump(data, f)
compressed_pickle("./py/similarity1", similarity1)
compressed_pickle("./py/similarity2", similarity2)
compressed_pickle("./py/similarity3", similarity3)
compressed_pickle("./py/similarity4", similarity4)
pickle.dump(all_movies, open('./py/movies.pkl', 'wb'))
# pickle.dump(similarity1, open('./py/similarity1.pkl', 'wb'))
# pickle.dump(similarity2, open('./py/similarity2.pkl', 'wb'))
# pickle.dump(similarity3, open('./py/similarity3.pkl', 'wb'))
# pickle.dump(similarity4, open('./py/similarity4.pkl', 'wb'))

In [ ]:
print(all_movies[all_movies['movie_id'] == 414906]['Title'])